In [201]:
import nltk
import os
import seaborn as sns
import re
from collections import Counter, OrderedDict
import math
import numpy as np

In [60]:
def edit_distance(s1 , s2):
    m = []
    for i in range(len(s1)+1):
        l = []
        for j in range(len(s2)+1):
            l.append([0])
        m.append(l)
    
    for i in range(len(s1) + 1):
        m[i][0] = i
        
    for j in range(len(s2) + 1):
        m[0][j] = j
        
    for i in range(1,len(s1) + 1):
        for j in range(1,len(s2) + 1):
            if s1[i-1] == s2[j-1]:
                m[i][j] = min(min(m[i-1][j-1], m[i-1][j] + 1), m[i][j-1] + 1)
            else:
                m[i][j] = min(min(m[i-1][j-1] + 1, m[i-1][j] + 1), m[i][j-1] + 1)
    
    return m[len(s1)][len(s2)]

In [68]:
qtoken = "gafffe"
compares = ["half", "puff", "gaffe", "cafe"]
temp = []
for i in compares :
    temp.append((i,edit_distance(qtoken,i)))

In [69]:
ans1 = sorted(temp, key= lambda x: x[1])

In [70]:
ans1

[('gaffe', 1), ('cafe', 3), ('half', 4), ('puff', 4)]

## 2

In [129]:
cranfield = open("C:\\Users\\acer\\Desktop\\lamaj\\Cranfield\\cran.all.1400", "r", encoding = "utf-8").read()

In [130]:
cranfield = re.sub(r"\n"," ", cranfield)

In [131]:
pattern = re.compile(r"\.W.+?\.I")
body = re.findall(pattern, cranfield)

In [132]:
len(body)

1399

In [133]:
docs = [i[3:-4] for i in body]

In [134]:
docs.append("""the buckling shear stress of simply-supported infinitely long plates with transverse stiffeners . this report is an extension of previous theoretical investigations of the elastic buckling in shear of flat plates reinforced by transverse stiffeners . the plates are treated as infinitely long and simply-supported along the long sides . stiffeners are spaced at regular intervals, dividing the plate into a number of panels of uniform size . the effect ob bending and torsional stiffnesses of the stiffener upon the buckling shear stress is calculated for the complete range of stiffnesses, for panels with ratios of width to stiffener spacing of graphical forms""")

In [135]:
stopwords = open('C:\\Users\\acer\\Desktop\\lamaj\\stopwords.txt').read()
stopwords = stopwords.split("\n")

In [322]:
def tokenize(text, lower = False, limit = 3):
    text = re.sub(r"\s+", " ", text)
    spept = re.compile(r"[^0-9a-zA-Z\-\']")
    text = re.sub(spept, " ", text)
    pt = re.compile(r"\w*[-|']{0,1}\w*")
    tokens = list(filter(pt.match, text.split(" ")))
    if lower:
        bt_tokens = [tk.lower() for tk in tokens if len(tk ) > limit and tk not in stopwords]
    else:
        bt_tokens = [tk for tk in tokens if len(tk ) > limit and tk not in stopwords]
    return bt_tokens

In [323]:
doc_id = 0
vocab = []
doc_tokens = []
for text in docs:
    token_list = tokenize(text,True, 3)
    doc_token = Counter(token_list)
    doc_tokens.append(doc_token)
    vocab.extend(doc_token.keys())
    doc_id = doc_id + 1

In [324]:
vocab = list(set(vocab))

In [325]:
len(vocab)

8518

In [326]:
def term_frequency(vocab, doc_tokens):
    tf_mat = []
    for cterm in vocab:
        tfrow = []
        for i in range(len(doc_tokens)):
            if doc_tokens[i].get(cterm) != None:
                tfrow.append(doc_tokens[i][cterm])
            else:
                tfrow.append(0)
        tf_mat.append(tfrow)

    return np.array(tf_mat)

In [327]:
def InverseDocFreq(vocab, doc_tokens):
    idf = []
    for cterm in vocab:
        count = 0
        for i in range(len(doc_tokens)):
            if doc_tokens[i].get(cterm) != None:
                count = count + 1
        idf.append(math.log(len(doc_tokens)/count))

    return np.array(idf)

In [328]:
def tfidfvector(vocab, doc_tokens):
    
    tf_mat = term_frequency(vocab, doc_tokens)
    idf = InverseDocFreq(vocab, doc_tokens)
    tfIdf = []
    for i in range(tf_mat.shape[0]):
        tfIdf.append(tf_mat[i,:].T*idf[i].T)

    return np.array(tfIdf)

In [329]:
def cosine(qvector, tfIdf):
    cos = {}
    for i in range(tfIdf.shape[1]):
        dot = (qvector.T*tfIdf[:,i].T).sum()
        cos[i] = dot/(tfIdf.shape[0])
    return cos

In [330]:
def queryCosineCalc(query, tfIdf):
    qtoken_list = tokenize(query,True,3)
    qvector = []
    for cterm in vocab:
        if cterm in qtoken_list:
            qvector.append(1)
        else:
            qvector.append(0)

    qvector = np.array(qvector)
    cos = cosine(qvector, tfIdf)
    ans2 = sorted([(i+1,v) for i,v in cos.items()], key = lambda x: x[1], reverse = True)
    print("Docs\t\tCosine : \n")
    for i in ans2[:4]:
        print(i)

In [331]:
tfIdf = tfidfvector(vocab, doc_tokens)

In [332]:
query = input("Enter Query : ")
queryCosineCalc(query, tfIdf)

Enter Query : what problems of heat conduction in composite slabs have been solved so far
Docs		Cosine : 

(144, 0.004675759368948767)
(542, 0.0035647364577836727)
(5, 0.0029318280320715607)
(707, 0.0027545549477447128)


In [333]:
query = input("Enter Query : ")
queryCosineCalc(query, tfIdf)

Enter Query : what is the basic mechanism of the transonic aileron buzz
Docs		Cosine : 

(496, 0.004617761486810521)
(199, 0.004244365087751186)
(1268, 0.0033079903539734736)
(903, 0.003023467465792699)


In [311]:
len(doc_tokens)

1400